In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# Функция загрузки данных TON с дополнительными признаками
def get_ton_data():
    ton = yf.Ticker("TON-USD")
    df = ton.history(period="7d", interval="1m")
    df.reset_index(inplace=True)
    df.rename(columns={"Datetime": "date"}, inplace=True)

    # Добавляем временные признаки
    df["hour"] = df["date"].dt.hour
    df["minute"] = df["date"].dt.minute
    df["dayofweek"] = df["date"].dt.dayofweek
    df["month"] = df["date"].dt.month

    # Добавляем лаговые признаки
    for lag in [1, 5, 10, 30, 60]:
        df[f"Close_lag{lag}"] = df["Close"].shift(lag)

    # Скользящее среднее
    df["SMA_10"] = df["Close"].rolling(window=10).mean()
    df["SMA_50"] = df["Close"].rolling(window=50).mean()

    # Волатильность
    df["Volatility"] = df["Close"].rolling(window=10).std()

    return df.dropna()

# Загружаем данные
data = get_ton_data()
data.set_index("date", inplace=True)

# Определяем признаки и целевую переменную
features = ["Open", "High", "Low", "Volume", "hour", "minute", "dayofweek", "month", "Close_lag1", "Close_lag5", "Close_lag10", "Close_lag30", "Close_lag60", "SMA_10", "SMA_50", "Volatility"]
target = "Close"
X = data[features]
y = data[target]

# Разделяем данные на тренировочные и тестовые
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Инициализация и обучение модели
model = RandomForestRegressor(n_estimators=300, max_depth=15, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

# Оценка точности
mae = mean_absolute_error(y_test, model.predict(X_test))
print(f"Средняя абсолютная ошибка: {mae:.5f} USD")

# Функция прогнозирования на несколько интервалов времени
def predict_future_prices(minutes_ahead=[10, 20, 30, 60]):
    global model, data, features
    last_row = pd.DataFrame([data.iloc[-1][features]], columns=features)
    prediction_results = []

    for future_time in minutes_ahead:
        future_index = data.index[-1] + pd.Timedelta(minutes=future_time)
        last_row["hour"] = future_index.hour
        last_row["minute"] = future_index.minute
        last_row["dayofweek"] = future_index.dayofweek
        last_row["month"] = future_index.month
        predicted_price = model.predict(last_row)[0]
        prediction_results.append((future_index, predicted_price))
        print(f"Прогноз на {future_time} минут ({future_index}): {predicted_price:.5f} USD")

    return prediction_results

# Выполняем прогноз
predictions = predict_future_prices()

# График
def plot_predictions(data, predictions):
    plt.figure(figsize=(14, 7))
    plt.plot(data.index[-500:], data["Close"].tail(500), label="Фактическая цена", color="blue", linewidth=2)
    times, prices = zip(*predictions)
    plt.scatter(times, prices, color="red", label="Прогноз", marker="x", s=120)
    plt.xlabel("Дата и время")
    plt.ylabel("Цена TON (USD)")
    plt.title("Прогноз цены TON с дополнительными признаками")
    plt.legend()
    plt.grid(True, linestyle="--", alpha=0.7)
    plt.xticks(rotation=45)
    plt.show()

# Построение графика
plot_predictions(data, predictions)
